https://signate.jp/competitions/141

In [8]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

## 観光宿泊者学習データ
全8国立公園（阿寒摩周国立公園、十和田八幡平国立公園、日光国立公園、伊勢志摩国立公園、
大山隠岐国立公園、阿蘇くじゅう国立公園、霧島錦江湾国立公園、慶良間諸島国立公園）の2015/01/01～2016/12/31までの観光宿泊者数データ。

In [35]:
df_train = pd.read_csv('../data/raw/train.tsv',sep='\t')
print('-- 行・列数 --')
print(df_train.shape)
print('------------')
print('-- カラム --')
print(df_train.columns)
print('------------')
print('-- データの概要 --')
print(df_train.info())
print('------------')
print('-- 欠損値の数 --')
print(df_train.isnull().sum())
print('------------')
print('-- 要約統計量 --')
print(df_train.describe())
print('------------')

-- 行・列数 --
(5848, 3)
------------
-- カラム --
Index(['datetime', 'park', 'visitors'], dtype='object')
------------
-- データの概要 --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5848 entries, 0 to 5847
Data columns (total 3 columns):
datetime    5848 non-null object
park        5848 non-null object
visitors    5848 non-null int64
dtypes: int64(1), object(2)
memory usage: 137.1+ KB
None
------------
-- 欠損値の数 --
datetime    0
park        0
visitors    0
dtype: int64
------------
-- 要約統計量 --
           visitors
count   5848.000000
mean    7186.736149
std     8791.087517
min        0.000000
25%      346.000000
50%     4214.500000
75%    11103.750000
max    87008.000000
------------


In [209]:
df_train.head(16)

,datetime,park,visitors
0,2015-01-01,阿寒摩周国立公園,11028
1,2015-01-01,十和田八幡平国立公園,34757
2,2015-01-01,日光国立公園,29714
3,2015-01-01,伊勢志摩国立公園,42652
4,2015-01-01,大山隠岐国立公園,3637
5,2015-01-01,阿蘇くじゅう国立公園,1369
6,2015-01-01,霧島錦江湾国立公園,35352
7,2015-01-01,慶良間諸島国立公園,151
8,2015-01-02,阿寒摩周国立公園,11153
9,2015-01-02,十和田八幡平国立公園,33795


In [ ]:
伊勢志摩国立公園、日光国立公園、霧島錦江湾国立公園の訪問者数が多い

In [36]:
# Feature engineering from datetime
# https://adtech.cyberagent.io/techblog/archives/259
# https://note.nkmk.me/python-pandas-datetime-timestamp/
df_train['datetime'] = pd.to_datetime(df_train['datetime']) # dtypeをdatetime64に変換
df_train['year'] = df_train['datetime'].dt.year
df_train['month'] = df_train['datetime'].dt.month
df_train['day'] = df_train['datetime'].dt.day
df_train['dayofweek'] = df_train['datetime'].dt.dayofweek
df_train.head()

,datetime,park,visitors,year,month,day,dayofweek
0,2015-01-01,阿寒摩周国立公園,11028,2015,1,1,3
1,2015-01-01,十和田八幡平国立公園,34757,2015,1,1,3
2,2015-01-01,日光国立公園,29714,2015,1,1,3
3,2015-01-01,伊勢志摩国立公園,42652,2015,1,1,3
4,2015-01-01,大山隠岐国立公園,3637,2015,1,1,3


In [265]:
## categorize park column
labels, uniques = pd.factorize(df_train['park'])
df_train['park'] = labels
df_train.head(16)

,datetime,park,visitors,year,month,day,dayofweek
0,2015-01-01,0,11028,2015,1,1,3
1,2015-01-01,1,34757,2015,1,1,3
2,2015-01-01,2,29714,2015,1,1,3
3,2015-01-01,3,42652,2015,1,1,3
4,2015-01-01,4,3637,2015,1,1,3
5,2015-01-01,5,1369,2015,1,1,3
6,2015-01-01,6,35352,2015,1,1,3
7,2015-01-01,7,151,2015,1,1,3
8,2015-01-02,0,11153,2015,1,2,4
9,2015-01-02,1,33795,2015,1,2,4


In [39]:
df_train['park'].unique()

array(['阿寒摩周国立公園', '十和田八幡平国立公園', '日光国立公園', '伊勢志摩国立公園', '大山隠岐国立公園',
       '阿蘇くじゅう国立公園', '霧島錦江湾国立公園', '慶良間諸島国立公園'], dtype=object)

In [81]:
# label encoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df_train['park'])
le.transform(['阿寒摩周国立公園', '十和田八幡平国立公園', '日光国立公園', '伊勢志摩国立公園','大山隠岐国立公園','阿蘇くじゅう国立公園', '霧島錦江湾国立公園', '慶良間諸島国立公園']) 

array([5, 1, 4, 0, 2, 6, 7, 3])

In [82]:
# onehot vector for park column
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
park_onehot = encoder.fit_transform(df_train['park'])
park_onehot
df_park = pd.DataFrame(park_onehot,columns=le.classes_)
df_train = pd.concat([df_train,df_park],axis=1)
df_train

,datetime,park,visitors,year,month,day,dayofweek,伊勢志摩国立公園,十和田八幡平国立公園,大山隠岐国立公園,...,阿蘇くじゅう国立公園,霧島錦江湾国立公園,伊勢志摩国立公園,十和田八幡平国立公園,大山隠岐国立公園,慶良間諸島国立公園,日光国立公園,阿寒摩周国立公園,阿蘇くじゅう国立公園,霧島錦江湾国立公園
0,2015-01-01,阿寒摩周国立公園,11028,2015,1,1,3,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2015-01-01,十和田八幡平国立公園,34757,2015,1,1,3,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,2015-01-01,日光国立公園,29714,2015,1,1,3,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2015-01-01,伊勢志摩国立公園,42652,2015,1,1,3,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2015-01-01,大山隠岐国立公園,3637,2015,1,1,3,0,0,1,...,0,0,0,0,1,0,0,0,0,0
5,2015-01-01,阿蘇くじゅう国立公園,1369,2015,1,1,3,0,0,0,...,1,0,0,0,0,0,0,0,1,0
6,2015-01-01,霧島錦江湾国立公園,35352,2015,1,1,3,0,0,0,...,0,1,0,0,0,0,0,0,0,1
7,2015-01-01,慶良間諸島国立公園,151,2015,1,1,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,2015-01-02,阿寒摩周国立公園,11153,2015,1,2,4,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,2015-01-02,十和田八幡平国立公園,33795,2015,1,2,4,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [30]:
# visitor数降順
df_train.sort_values('visitors',ascending=False)

,datetime,park,visitors,year,month,day,dayofweek
3907,2016-05-03,伊勢志摩国立公園,87008,2016,5,3,1
3915,2016-05-04,伊勢志摩国立公園,84174,2016,5,4,2
3891,2016-05-01,伊勢志摩国立公園,64150,2016,5,1,6
2915,2015-12-31,伊勢志摩国立公園,60708,2015,12,31,3
2907,2015-12-30,伊勢志摩国立公園,58852,2015,12,30,2
3906,2016-05-03,日光国立公園,57547,2016,5,3,1
2931,2016-01-02,伊勢志摩国立公園,57421,2016,1,2,5
987,2015-05-04,伊勢志摩国立公園,56153,2015,5,4,0
979,2015-05-03,伊勢志摩国立公園,55704,2015,5,3,6
5843,2016-12-31,伊勢志摩国立公園,54754,2016,12,31,5


日付ごとの平均 visitor数は、GWと年末年始が多い
単純に休日フラグをつけたデータが欲しいかも

In [113]:
# 日付ごとの平均 visitor数
df_train.groupby('datetime').mean().sort_values('visitors',ascending=False)

,visitors
datetime,
2016-05-03,28594.375
2015-05-03,25773.000
2016-12-31,25520.750
2016-05-04,25445.875
2015-05-04,25400.750
2015-12-31,25056.875
2015-09-20,23663.000
2015-09-21,22938.625
2016-01-01,21294.500


## 予測対象データ（test.tsv）
全8国立公園（阿寒摩周国立公園、十和田八幡平国立公園、日光国立公園、伊勢志摩国立公園、大山隠岐国立公園、阿蘇くじゅう国立公園、霧島錦江湾国立公園、慶良間諸島国立公園）の2017/01/01～2017/12/31までの予測対象データ。  
予測対象なのでtrainデータと異なり、visitorsカラムが無い。
  
※indexは予測結果投稿時に必要となります。応募用サンプルファイルの説明もご参照ください。

In [62]:
df_test = pd.read_csv('../data/raw/test.tsv',sep='\t')
df_test['datetime'] = pd.to_datetime(df_test['datetime']) # dtypeをdatetime64に変換
df_test['year'] = df_test['datetime'].dt.year
df_test['month'] = df_test['datetime'].dt.month
df_test['day'] = df_test['datetime'].dt.day
df_test['dayofweek'] = df_test['datetime'].dt.dayofweek

#labels, uniques = pd.factorize(df_test['park'])
#df_test['park'] = labels

# onehot vector for park column
encoder = LabelBinarizer()
park_onehot_test = encoder.fit_transform(df_test['park'])
df_park_test = pd.DataFrame(park_onehot_test,columns=le.classes_)
df_test = pd.concat([df_test,df_park_test],axis=1)
df_test.head(16)

,index,datetime,park,year,month,day,dayofweek,伊勢志摩国立公園,十和田八幡平国立公園,大山隠岐国立公園,慶良間諸島国立公園,日光国立公園,阿寒摩周国立公園,阿蘇くじゅう国立公園,霧島錦江湾国立公園
0,0,2017-01-01,阿寒摩周国立公園,2017,1,1,6,0,0,0,0,0,1,0,0
1,1,2017-01-01,十和田八幡平国立公園,2017,1,1,6,0,1,0,0,0,0,0,0
2,2,2017-01-01,日光国立公園,2017,1,1,6,0,0,0,0,1,0,0,0
3,3,2017-01-01,伊勢志摩国立公園,2017,1,1,6,1,0,0,0,0,0,0,0
4,4,2017-01-01,大山隠岐国立公園,2017,1,1,6,0,0,1,0,0,0,0,0
5,5,2017-01-01,阿蘇くじゅう国立公園,2017,1,1,6,0,0,0,0,0,0,1,0
6,6,2017-01-01,霧島錦江湾国立公園,2017,1,1,6,0,0,0,0,0,0,0,1
7,7,2017-01-01,慶良間諸島国立公園,2017,1,1,6,0,0,0,1,0,0,0,0
8,8,2017-01-02,阿寒摩周国立公園,2017,1,2,0,0,0,0,0,0,1,0,0
9,9,2017-01-02,十和田八幡平国立公園,2017,1,2,0,0,1,0,0,0,0,0,0


## SNSデータ（hotlink.tsv）
観光及び国立公園に関連する語句が含まれる投稿の2015/01/01～2017/12/30までの日別件数（ブログ・twitter・掲示板）データ。

In [49]:
df_hotlink = pd.read_csv('../data/raw/hotlink.tsv',sep='\t')
print(df_hotlink.shape)
df_hotlink.head()

(144540, 4)


,datetime,domain,keyword,count
0,2015-01-01,twitter_sampling,国立公園,148
1,2015-01-01,blog,アオウミガメ,11
2,2015-01-01,twitter_sampling,アオウミガメ,1
3,2015-01-01,bbs,阿波連岬園地,0
4,2015-01-01,blog,阿波連岬園地,0


## ロケーション付SNSデータ（nightley.tsv）
日光国立公園区画内のSNSロケーション解析データ投稿数データ。  
※１ 本データは日光国立公園区画内のみ含まれています。  
※２ データ期間は2016/01/01～2017/12/30までです。

日本人と外国人のカウント数がある。

In [83]:
df_nightley = pd.read_csv('../data/raw/nightley.tsv',sep='\t')
df_nightley.head(100)

,datetime,Japan_count,Foreign_count
0,2016-01-01,28,0
1,2016-01-02,66,1
2,2016-01-03,75,6
3,2016-01-04,50,1
4,2016-01-05,25,0
5,2016-01-06,29,2
6,2016-01-07,7,1
7,2016-01-08,9,1
8,2016-01-09,43,2
9,2016-01-10,68,2


## メッシュ型流動人口データ（agoop.zip）
各国立公園付近エリア単位の「平均人口」の集計値データ。zipファイル内には100mメッシュ集計データ、100mメッシュIDデータ、都道府県・市区町村マスタの3種類のデータが同封されています。

100mメッシュ集計データ（month_time_mesh100m_YYYYMM.tsv, 201501～201711までの全35ファイル）
※流動人口数が1人未満の場合はデータが出力されていませんのでご注意ください。

In [84]:
df_pop_city_2015 = pd.read_csv('../data/raw/agoop/city_2017.tsv',sep='\t')
df_pop_city_2015.head(100)

,prefcode,prefecture,citycode,cwtv,address
0,1,北海道,1000,NaN,北海道
1,1,北海道,1100,札幌市,北海道札幌市
2,1,北海道,1101,札幌市中央区,北海道札幌市中央区
3,1,北海道,1102,札幌市北区,北海道札幌市北区
4,1,北海道,1103,札幌市東区,北海道札幌市東区
5,1,北海道,1104,札幌市白石区,北海道札幌市白石区
6,1,北海道,1105,札幌市豊平区,北海道札幌市豊平区
7,1,北海道,1106,札幌市南区,北海道札幌市南区
8,1,北海道,1107,札幌市西区,北海道札幌市西区
9,1,北海道,1108,札幌市厚別区,北海道札幌市厚別区


In [27]:
df_pop_mesh100m_2015 = pd.read_csv('../data/raw/agoop/mesh100m_2015.tsv',sep='\t')
df_pop_mesh100m_2015.head()

,mesh100mid,loncenter,latcenter,lonmax,latmax,lonmin,latmin,prefcode,citycode
0,39271198141,127.226875,26.162083,127.22750,26.162500,127.22625,26.161667,47,47354
1,39271198203,127.235625,26.158750,127.23625,26.159167,127.23500,26.158333,47,47354
2,39271198210,127.231875,26.159583,127.23250,26.160000,127.23125,26.159167,47,47354
3,39271198212,127.234375,26.159583,127.23500,26.160000,127.23375,26.159167,47,47354
4,39271198214,127.236875,26.159583,127.23750,26.160000,127.23625,26.159167,47,47354


In [28]:
df_pop_month_time_mesh100m_201502 = pd.read_csv('../data/raw/agoop/month_time_mesh100m_201502.tsv',sep='\t')
df_pop_month_time_mesh100m_201502.head()

,park,mesh100mid,year,month,dayflag,hour,population
0,伊勢志摩国立公園,51362368112,2015,2,1,21,2
1,伊勢志摩国立公園,51362368112,2015,2,2,21,1
2,伊勢志摩国立公園,51362368212,2015,2,1,16,33
3,伊勢志摩国立公園,51362368212,2015,2,1,17,36
4,伊勢志摩国立公園,51362368212,2015,2,1,18,36


## 国別月別来訪者数集計データ
十和田八幡平、日光、阿寒摩周、伊勢志摩国立公園周辺の国籍別・月単位での来訪者数集計データ。  
※１ 本データには上記４つの国立公園周辺のデータのみ含まれています。  
※２ データ期間は2016年1月～2017年11月までとなります。  
※３ countが1以上10未満の値の場合は「1-9」という値となっています。

★どこの国の来訪者が多いか調べて、日本人の来訪者数と比較する  
多いところを予測するのが良さそう  
中国の大型連休などに合わせて来訪者が増えそう

In [96]:
df_colopl = pd.read_csv('../data/raw/colopl.tsv',sep='\t')
# countが1-9の場合、0を入れる。それ以外はそのまま。 ここ本当は9ではなく何を代入すればいいか考える
df_colopl['count'] = [0 if i=='1-9' else i for i in df_colopl['count']] 
df_colopl['count'] = df_colopl['count'].astype('int64')
print(df_colopl.shape)
print(df_colopl.describe())
print(df_colopl.info())
df_colopl.head(30)

(1701, 5)
              year        month        count
count  1701.000000  1701.000000  1701.000000
mean   2016.485009     6.403292     4.822457
std       0.499922     3.139627    14.812165
min    2016.000000     1.000000     0.000000
25%    2016.000000     4.000000     0.000000
50%    2016.000000     6.000000     0.000000
75%    2017.000000     9.000000     0.000000
max    2017.000000    12.000000   145.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701 entries, 0 to 1700
Data columns (total 5 columns):
year          1701 non-null int64
month         1701 non-null int64
park          1701 non-null object
country_jp    1701 non-null object
count         1701 non-null int64
dtypes: int64(3), object(2)
memory usage: 66.5+ KB
None


,year,month,park,country_jp,count
0,2016,1,阿寒摩周国立公園,アメリカ合衆国,0
1,2016,1,阿寒摩周国立公園,シンガポール,0
2,2016,1,阿寒摩周国立公園,マレーシア,0
3,2016,1,阿寒摩周国立公園,中国,0
4,2016,1,阿寒摩周国立公園,台湾,19
5,2016,1,阿寒摩周国立公園,朝鮮、南部,0
6,2016,1,阿寒摩周国立公園,香港(PRC),12
7,2016,1,十和田八幡平国立公園,アメリカ合衆国,0
8,2016,1,十和田八幡平国立公園,オーストラリア,0
9,2016,1,十和田八幡平国立公園,ノルウェー,0


In [108]:
# 国ごとの来訪者数平均を多い順に
df_colopl.groupby('country_jp').mean().sort_values('count', ascending=False)['count']

country_jp
台湾             36.413043
アメリカ合衆国        19.402174
タイ             17.177215
香港(PRC)         5.466667
スペイン            4.090909
フランス            2.844444
ドイツ             2.654545
オーストラリア         2.608108
イタリア            2.593750
英国              2.276923
シンガポール          2.050633
マレーシア           0.835821
インドネシア          0.794872
中国              0.743902
オランダ            0.617647
ブラジル            0.434783
フィリピン           0.423077
メキシコ            0.416667
スイス             0.371429
ルーマニア           0.000000
ベラルーシ           0.000000
ベトナム            0.000000
ブルネイ・ダルサラーム     0.000000
ブルガリア           0.000000
南アフリカ           0.000000
再結合(フランス)       0.000000
ロシア連邦           0.000000
フィンランド          0.000000
パレスチナ           0.000000
朝鮮、南部           0.000000
                 ...    
ウクライナ           0.000000
インド             0.000000
イラン             0.000000
イスラエル           0.000000
アルバニア           0.000000
アルゼンチン共和国       0.000000
アルジェリア          0.000000
アラブ首長国連邦        0.000000
キプロス          

# 気象庁データ（weather.tsv）
国立公園周辺12地点の気象観測データ。

In [33]:
df_weather = pd.read_csv('../data/raw/weather.tsv',sep='\t')
df_weather.head()

,年月日,地点,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),10分間降水量の最大(mm),日照時間(時間),合計全天日射量(MJ/㎡),最深積雪(cm),...,最多風向(16方位),平均蒸気圧(hPa),平均現地気圧(hPa),平均湿度(%),平均海面気圧(hPa),最小相対湿度(%),最低海面気圧(hPa),平均雲量(10分比),天気概況(昼:06時~18時),天気概況(夜:18時~翌日06時)
0,2015/1/1,十和田,-1.9,0.3,-2.9,5.0,1.0,2.1,NaN,18.0,...,西南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/1/2,十和田,-3.9,-1.1,-8.0,0.0,0.0,3.3,NaN,17.0,...,西北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/1/3,十和田,-4.0,-1.6,-8.9,2.0,0.5,1.4,NaN,21.0,...,西南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015/1/4,十和田,0.6,3.4,-1.6,0.0,0.0,2.6,NaN,19.0,...,南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/1/5,十和田,0.8,5.4,-5.1,0.0,0.0,4.0,NaN,14.0,...,西南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 積雪気象観測データ（nied_oyama.tsv）
防災科学技術研究所 ⼤⼭鏡ヶ成積雪気象観測施設における積雪気象観測データ。  
緯度経度35°20′21″ 133°34′51″  
標高：875m  

In [34]:
df_nied_oyama = pd.read_csv('../data/raw/nied_oyama.tsv',sep='\t')
print(df_train.shape)
print(df_train.columns)
df_nied_oyama.head()

,日時,積雪深(cm),積雪深重量(kg m^-2),気温(℃),降水量(mm h-1)
0,2015-01-01 00:00:00,99.3,305.1,-4.8,1.0
1,2015-01-01 01:00:00,101.9,308.3,-4.8,1.0
2,2015-01-01 02:00:00,102.5,310.3,-5.3,0.0
3,2015-01-01 03:00:00,101.8,310.6,-5.5,0.0
4,2015-01-01 04:00:00,101.9,311.1,-6.1,0.5


## 路線検索データ（jorudan.zip）
ジョルダンの路線検索ログデータ。

In [36]:
df_train = pd.read_csv('../data/raw/jorudan.tsv',sep='\t')
print(df_train.shape)
print(df_train.columns)
df_train.head()

,access_date,access_time,departure_and_arrival_type,departure_and_arrival_date,departure_and_arrival_time,departure_and_arrival_place_type,park,departure_prefecture,arrival_prefecture
0,2015-01-02,17:19:52,D,2015-01-03,06:44,D,日光国立公園,栃木県,神奈川
1,2015-01-02,15:16:57,A,2015-01-03,06:40,D,日光国立公園,栃木県,神奈川
2,2015-01-02,15:17:22,D,2015-01-03,06:40,D,日光国立公園,栃木県,神奈川
3,2015-01-02,15:35:05,D,2015-01-03,06:40,D,日光国立公園,栃木県,神奈川
4,2015-01-02,17:07:22,D,2015-01-03,07:57,D,日光国立公園,栃木県,神奈川


## 応募用サンプルファイル（sample_submit.tsv）
1列目の値はtest.tsvの"index"に対応した値であり、2列目が"index"に対応する予測値となります。ヘッダ無しtsv形式で投稿ください。

In [316]:
df_sample_submit = pd.read_csv('../data/raw/sample_submit.tsv',sep='\t')
df_sample_submit.head()

,0,53822
0,1,48553
1,2,40052
2,3,1524
3,4,10759
4,5,57010


In [55]:
# 
df_holiday = pd.read_csv('../data/raw/holiday.csv')
df_holiday['datetime'] = pd.to_datetime(df_holiday['datetime'])
df_holiday.columns

Index(['datetime', 'holiday'], dtype='object')

In [56]:
# 学習データと休日データの結合
df_merged = pd.merge(df_train,df_holiday,on='datetime')
df_merged

,datetime,park,visitors,year,month,day,dayofweek,伊勢志摩国立公園,十和田八幡平国立公園,大山隠岐国立公園,慶良間諸島国立公園,日光国立公園,阿寒摩周国立公園,阿蘇くじゅう国立公園,霧島錦江湾国立公園,holiday
0,2015-01-01,阿寒摩周国立公園,11028,2015,1,1,3,0,0,0,0,0,1,0,0,1
1,2015-01-01,十和田八幡平国立公園,34757,2015,1,1,3,0,1,0,0,0,0,0,0,1
2,2015-01-01,日光国立公園,29714,2015,1,1,3,0,0,0,0,1,0,0,0,1
3,2015-01-01,伊勢志摩国立公園,42652,2015,1,1,3,1,0,0,0,0,0,0,0,1
4,2015-01-01,大山隠岐国立公園,3637,2015,1,1,3,0,0,1,0,0,0,0,0,1
5,2015-01-01,阿蘇くじゅう国立公園,1369,2015,1,1,3,0,0,0,0,0,0,1,0,1
6,2015-01-01,霧島錦江湾国立公園,35352,2015,1,1,3,0,0,0,0,0,0,0,1,1
7,2015-01-01,慶良間諸島国立公園,151,2015,1,1,3,0,0,0,1,0,0,0,0,1
8,2015-01-02,阿寒摩周国立公園,11153,2015,1,2,4,0,0,0,0,0,1,0,0,0
9,2015-01-02,十和田八幡平国立公園,33795,2015,1,2,4,0,1,0,0,0,0,0,0,0


In [51]:
import pandas as pd
from sklearn.cross_validation import train_test_split
import math
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, roc_curve
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [69]:
df_except_y = df_merged.drop(['park','visitors','datetime'], axis=1)
X_train = df_except_y.as_matrix() 
y_train = df_merged['visitors'].as_matrix() # 目的変数に "y" を利用
df_except_y.head()

array([[2015,    1,    1, ...,    0,    0,    1],
       [2015,    1,    1, ...,    0,    0,    1],
       [2015,    1,    1, ...,    0,    0,    1],
       ...,
       [2016,   12,   31, ...,    1,    0,    1],
       [2016,   12,   31, ...,    0,    1,    1],
       [2016,   12,   31, ...,    0,    0,    1]])

In [71]:
df_test_merged = pd.merge(df_test,df_holiday,on='datetime')
df_test_merged = df_test_merged.drop(['park','datetime','index'], axis=1)
X_test = df_test_merged.as_matrix()
df_test_merged.head()

,year,month,day,dayofweek,伊勢志摩国立公園,十和田八幡平国立公園,大山隠岐国立公園,慶良間諸島国立公園,日光国立公園,阿寒摩周国立公園,阿蘇くじゅう国立公園,霧島錦江湾国立公園,holiday
0,2017,1,1,6,0,0,0,0,0,1,0,0,1
1,2017,1,1,6,0,1,0,0,0,0,0,0,1
2,2017,1,1,6,0,0,0,0,1,0,0,0,1
3,2017,1,1,6,1,0,0,0,0,0,0,0,1
4,2017,1,1,6,0,0,1,0,0,0,0,0,1


In [72]:
params = {"learning_rate":[0.1,0.3,0.5],
        "max_depth": [2,3,5,10],
         "subsample":[0.5,0.8,0.9,1],
         "colsample_bytree": [0.5,1.0],
         }

# モデルにインスタンス生成
model = xgb.XGBRegressor()
cv = GridSearchCV(model,params,cv=10,n_jobs=-1)

In [73]:
# 予測モデルを作成
cv.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.1, 0.3, 0.5], 'max_depth': [2, 3, 5, 10], 'subsample': [0.5, 0.8, 0.9, 1], 'colsample_bytree': [0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [76]:
# predict
y_train_pred = cv.predict(X_train)
y_test_pred = cv.predict(X_test)

In [77]:
submit = pd.DataFrame({
    '':df_test['index'],
    '':y_test_pred})
submit.to_csv('../submit/submit_3.tsv',sep='\t')

In [339]:
submit = pd.DataFrame({
    '':df_train,
    '':y_train_pred})
submit.to_csv('../submit/validation.csv')

In [357]:
df = pd.read_csv('../submit/submit_2.csv',index_col=0)
df

,10313.23
0,
1.0,24170.57000
2.0,32503.21000
3.0,46129.73000
4.0,1406.22000
5.0,645.75000
6.0,23568.56000
7.0,877.77000
8.0,9738.62000
9.0,22452.64000


In [ ]:
df.to_csv('../submit/submit_2.tsv',sep='\t')